In [1]:
import io, math, numpy, sys
import numpy as np
import scipy.optimize as opt
from scipy import stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd
import matplotlib.pyplot as plt
import random
from scipy.spatial import ConvexHull, convex_hull_plot_2d

In [2]:
from wiresharkoui import wiresharkoui

In [7]:
## Constant Constants
AREA_EARTH_SQ_KM=510100000.0 # 510.1 million km²
RADIAN_2_KM=6371.0
SQ_RADIAN_2_SQ_KM=AREA_EARTH_SQ_KM/(4.0*math.pi)

In [8]:
# Unit sphere radian utility functions
def xyz2angles(xyz):
    lon_radians = numpy.arctan2(xyz[1], xyz[0])
    lat_radians = numpy.arcsin(xyz[2])
    return np.array([lat_radians, lon_radians])

def angles2xyz(latlon_radians):
    xyz = [math.cos(latlon_radians[0]) * math.cos(latlon_radians[1]), math.cos(latlon_radians[0]) * math.sin(latlon_radians[1]), math.sin(latlon_radians[0])]
    return np.array(xyz)


In [9]:
# Unit sphere lat lon utility functions
def degrees_to_radians(angle):
    rad = math.pi * angle / 180.0
    return rad
np_degrees_to_radians = np.vectorize(degrees_to_radians)

def radians_to_degrees(rad):
    deg = 180.0 * rad / math.pi
    return deg
np_radians_to_degrees = np.vectorize(radians_to_degrees)

def xyz2latlon(xyz):
    latlon_radians = xyz2angles(xyz)
    lat_deg = radians_to_degrees(latlon_radians[0])
    lon_deg = radians_to_degrees(latlon_radians[1])
    return np.array([lat_deg, lon_deg])
    
def latlon2xyz(latlon_deg):
    xyz = angles2xyz([degrees_to_radians(latlon_deg[0]), degrees_to_radians(latlon_deg[1])])
    return xyz


In [3]:
SAMPLE_DATA_PATH="db.test-tracy.stockholm.psv"
def loadSampleCity():
    D = {}
    with open(SAMPLE_DATA_PATH) as f:
        for line in f:
            line = line.rstrip()
            A = line.split('|')
            if len(A) == 6:
                (bssid, ssid, lat, lon, r, device) = A
                short_name, long_name, approx = wiresharkoui.lookupOUI(bssid, approx=True)
                if short_name not in D:
                    D[short_name] = []
                D[short_name].append( np.array([lat, lon], dtype=float) )
    return D

In [4]:
# Second Best Friend (on unit sphere):
#  Finds the distance of the second-closest point for every point in the dataset
#  Slower than HullPies, but very consistent
#  Gives a direct measure of the "grid" spacing and the consistency of this spacing.
def secondBestFriendUS(XYZ):
    N = len(XYZ)
    # Space for distances to nearest and second-nearest neighbors
    secondBestFriendDist = np.zeros((N, 2))
    
    def dist(iPt, jPt):
        D = math.pi * (1.0 - np.dot(XYZ[iPt], XYZ[jPt]))
        return D

    for iPt in range(N):
        for jPt in range(iPt+1, N, 1):
            d = dist(iPt, jPt)
            if secondBestFriendDist[iPt][0] == 0.0:
                secondBestFriendDist[iPt,0] = d
            elif d < secondBestFriendDist[iPt,0]:
                secondBestFriendDist[iPt,1] = secondBestFriendDist[iPt,0]
                secondBestFriendDist[iPt,0] = d
            elif secondBestFriendDist[iPt][1] == 0.0:
                secondBestFriendDist[iPt,1] = d
            elif d < secondBestFriendDist[iPt,1]:
                secondBestFriendDist[iPt,1] = d

            if secondBestFriendDist[jPt][0] == 0.0:
                secondBestFriendDist[jPt,0] = d
            elif d < secondBestFriendDist[jPt,0]:
                secondBestFriendDist[jPt,1] = secondBestFriendDist[jPt,0]
                secondBestFriendDist[jPt,0] = d
            elif secondBestFriendDist[jPt][1] == 0.0:
                secondBestFriendDist[jPt,1] = d
            elif d < secondBestFriendDist[jPt,1]:
                secondBestFriendDist[jPt,1] = d
    #

    sbfMin = min(secondBestFriendDist[: ,1])
    sbfAvg = np.sum(secondBestFriendDist[: ,1], axis=0)/N
    sbfMax = max(secondBestFriendDist[: ,1])
    sbfStdD = np.std((secondBestFriendDist[: ,1]-sbfMin)/(sbfAvg-sbfMin))
    sbfMedian = np.median(secondBestFriendDist[: ,1], axis=0)
    
    return sbfAvg, sbfStdD, (sbfMin, sbfMedian, sbfMax)
#

def secondBestFriendLatLon(LATLON):
    N = len(LATLON)
    XYZ = [ latlon2xyz(LATLON[i]) for i in range(N) ]
    sbfAvg, sbfStdD, (sbfMin, sbfMedian, sbfMax) =  secondBestFriendUS(XYZ)
    # convert radian distances to kilometers (everything but normalized StdDev)
    sbfAvg = sbfAvg * RADIAN_2_KM
    sbfMin = sbfMin * RADIAN_2_KM
    sbfMedian = sbfMedian * RADIAN_2_KM
    sbfMax = sbfMax * RADIAN_2_KM
    return sbfAvg, sbfStdD, (sbfMin, sbfMedian, sbfMax)
    

In [10]:
def testSecondBestFriendLatLon():
    testData = loadSampleCity()
    for tag in testData:
        print([tag, secondBestFriendLatLon(testData[tag])])

testSecondBestFriendLatLon()

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in true_divide


['Cc&CTech', (0.0, nan, (0.0, 0.0, 0.0))]
['D-Link', (2.6417417770134702e-06, 10.943295425516812, (2.6665452240975592e-11, 3.4453541992156533e-07, 0.0009107033949122553))]
['Apple', (4.980403941912167e-07, 9.25238132199686, (-4.444242040162599e-12, 5.654186935596866e-08, 0.00035763311674600115))]
['Cisco-Li', (2.552999948808767e-06, 3.9058866964327983, (4.444242040162599e-12, 5.666686366334823e-07, 0.00024358951508247152))]
['', (3.824483721060067e-05, 8.255736190324038, (1.1621692935025196e-09, 2.9999167080142356e-06, 0.003840872990532957))]
['ThomsonT', (2.1744999006972802e-06, 5.103531566354155, (-4.444242040162599e-12, 3.999973384617744e-07, 0.00033956771283270214))]
['Vsoontec', (0.00014782590839244558, 2.0747373438756984, (1.164480299363404e-07, 7.811387801228082e-05, 0.0017120090454634162))]
['Cisco', (7.141746285086089e-07, 19.563223134424383, (-4.444242040162599e-12, 8.690715309537962e-09, 0.0010491984718702177))]
['ChinaMob', (3.0017921275038967e-05, 1.5208622293438896, (2.97

['SinkyoEl', (3.4796991993394485e-06, 4.054898623556884, (4.9219980594800775e-09, 1.002097694760453e-06, 0.00025841917747237915))]
['VivoMobi', (0.00011616837526525618, 0.8021613823466898, (2.105237454425023e-08, 0.00010187548028118233, 0.00023210779132838022))]
['ExtremeN', (1.2991661516853688e-05, 7.483542670661824, (4.444242040162599e-12, 2.6118810470035592e-08, 0.0011923829574804882))]
['Axesstel', (2.3048805541353402e-05, 1.5066639658844938, (1.337716854088942e-08, 6.991099381876539e-06, 0.0002443599199958917))]
['Fraunhof', (7.34876149563526e-05, 2.4767355770580655, (3.6003982491497853e-06, 1.6895958284516213e-05, 0.0010038231206237628))]
['M2MSolut', (3.6088308385807515e-05, 2.9294484474201896, (1.3021629177676413e-09, 9.146896755871471e-06, 0.0011562786551134179))]
['TexasIns', (3.063970375431044e-05, 2.6617022624204627, (8.475169570590076e-09, 1.245082628488507e-05, 0.0008603999725495724))]
['ZygateCo', (6.043789478833465e-05, 1.946872238018127, (1.545085187682929e-07, 1.66762

['YotaDevi', (3.0920168037228065e-06, 4.454069475900773, (1.7199216695429258e-09, 9.322286545865468e-07, 0.00028817250241558586))]
['Hangzhou', (7.196855689736441e-06, 3.3410461305222237, (4.444242040162599e-12, 1.674314857726777e-06, 0.0004151998083173421))]
['Lite-OnT', (0.0, nan, (0.0, 0.0, 0.0))]
['FuzeEnte', (1.1809806872154276e-05, 2.7308488662281554, (1.1777241406430885e-10, 1.072368938838994e-06, 0.00024155993862665925))]
['CompalBr', (8.252570674193968e-07, 9.02110523360644, (2.2221210200812993e-12, 4.456019281569029e-08, 0.00038794127419822786))]
['USRoboti', (0.00030996594260339507, 1.7322836368024652, (3.382768160685063e-06, 3.624159722535668e-05, 0.0018552220608854637))]
['Devolo', (6.784101051374508e-05, 2.8943369960372776, (1.6270370109035274e-08, 1.678172459817638e-05, 0.0011632631459073807))]
['SitecomE', (0.0002673130866857443, 1.2764790324609887, (6.306823879194743e-06, 0.00011535392420968296, 0.001072139980377569))]
['Khwahish', (2.9726145784876444e-06, 3.5933406641

['WebastoS', (3.8262055712354005e-06, 12.38928016588379, (4.444242040162599e-12, 4.7523391195968706e-07, 0.0016577512476394479))]
['Blackber', (0.00023119019237788287, 1.153470824269216, (2.4185031873520045e-05, 0.0001156906199866457, 0.0007688018930932278))]
['RanovusU', (0.0013913381225690573, 1.559705807423918, (0.001344240681418474, 0.001351319465695803, 0.0015184728774661487))]
['Micro-St', (0.0, nan, (0.0, 0.0, 0.0))]
['WesternD', (0.0011365034149045908, 1.0628414600224698, (0.0005628716273385968, 0.0007865868908881844, 0.00208237795773816))]
['Teltonik', (0.000244376035175936, 1.7224338519201423, (8.191626928427702e-08, 3.77562293555199e-05, 0.0016235846214691621))]
['Yamaichi', (0.0005553659806500267, 1.0640933148897553, (0.000365175875448081, 0.0005045008301664451, 0.0008860916735825801))]
['Bridgeco', (0.0, nan, (0.0, 0.0, 0.0))]
['Kaloom', (0.0001286666464881201, 1.6995451573317348, (1.9236901670843808e-08, 4.1971080731718994e-05, 0.000954852442008326))]
['VoyetraT', (0.0, n

In [11]:
# Now that I understand the data, we are not really trying to find what I thought we were trying to find:
# The most regular grid of a set of grids.


In [12]:
# What is really desired is to know which items' circles best
# interact with a path.  

In [13]:
# First step:  Lets put this in KML where we can draw paths?